In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 행동 인식 모델 구축
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(30, 33)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3개의 행동 분류 (달리기, 걷기, 점프)

# 모델 컴파일
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# 웹캠에서 비디오 캡처
cap = cv2.VideoCapture(0)
sequence = []
predictions = []
actions = ['walking', 'running', 'jumping']

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 이미지 전처리
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # 포즈 추정
    results = pose.process(image)
    
    # 이미지 원래대로 변경
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 랜드마크 추출
    if results.pose_landmarks:
        keypoints = []
        for lm in results.pose_landmarks.landmark:
            keypoints.append([lm.x, lm.y, lm.z])
        keypoints = np.array(keypoints).flatten()
        
        # 시퀀스 업데이트
        sequence.append(keypoints)
        if len(sequence) == 30:
            prediction = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(prediction))
            sequence = []
            
            # 예측 결과 출력
            if predictions:
                action = actions[predictions[-1]]
                cv2.putText(frame, f'Action: {action}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 포즈 랜드마크 그리기
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # 결과 이미지 표시
    cv2.imshow('Action Recognition', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


ValueError: in user code:

    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\yunji\anaconda3\envs\poseEnv\lib\site-packages\keras\src\engine\